# Flask and Databases

Source: https://pythonbasics.org/flask-sqlite/

Python has built-in support for SQLite. The SQlite3 module comes with the Python release. In this article you will learn how the Flask application interacts with SQLite.

SQLite is a relational database system that uses the SQL query language to interact with the database. Each database can have tables and each table can have records.

In [6]:
# Create a database
import sqlite3

conn = sqlite3.connect('database.db')
print ("Opened database successfully")

conn.execute('CREATE TABLE students (name TEXT PRIMARY KEY, addr TEXT, city TEXT, pin TEXT)')
print ("Table created successfully")
conn.close()

Opened database successfully
Table created successfully


In [7]:
from flask import Flask, render_template, request
import sqlite3 as sql


app = Flask(__name__)

@app.route('/')
def home():
    return render_template('student.html')



@app.route('/addrec',methods = ['POST', 'GET'])
def addrec():
    if request.method == 'POST':
        try:
            nm = request.form['nm']
            addr = request.form['add']
            city = request.form['city']
            pin = request.form['pin']
         
            with sql.connect("database.db") as con:
                cur = con.cursor()
            
                cur.execute("INSERT INTO students (name,addr,city,pin) VALUES (?,?,?,?)",(nm,addr,city,pin) ) # ? and tuple for placeholders
            
                con.commit()
                msg = "Record successfully added"
        except:
            con.rollback()
            msg = "error in insert operation"
      
        finally:
            return render_template("result.html",msg = msg)
            con.close()

@app.route('/list')
def list_all():
    con = sql.connect("database.db")
    con.row_factory = sql.Row
    
    cur = con.cursor()
    cur.execute("select * from students")
    
    rows = cur.fetchall() # returns list of dictionaries
    return render_template("list.html",rows = rows)

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Dec/2020 21:19:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2020 21:19:40] "POST /addrec HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2020 21:19:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2020 21:19:42] "GET /list HTTP/1.1" 200 -


# Notes

If a `finally` clause is present, the finally clause will execute as the last task before the try statement completes. The finally clause runs whether or not the try statement produces an exception

----

`con.row_factory = sqlite3.Row`
Sets the row_factory to the callable sqlite3.Row, which converts the plain tuple into a more useful object.

So now, when you pull rows from your database, you won't get back a plain python tuple, but a special object that makes it easier to work with (e.g. allowing you to access columns using names whereas a plain tuple would make you use numbered indices).

Creates a 'dictionary cursor', - instead of tuples it starts returning 'dictionary' rows after fetchall or fetchone.